In [1]:
from docx import Document
from docx.shared import Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
from typing import Dict, List, Tuple, Union
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def validate_dictionary_structure(dict_data: Dict, required_fields: List[str], dict_name: str) -> bool:
    """Validate dictionary structure and required fields."""
    if not isinstance(dict_data, dict):
        raise ValueError(f"{dict_name} must be a dictionary")
    
    missing_fields = [field for field in required_fields if field not in dict_data]
    if missing_fields:
        raise ValueError(f"Missing required fields in {dict_name}: {missing_fields}")
    
    return True

def validate_input_data(
    employee_data: Dict,
    earnings_data: List[Tuple[str, str]],
    deductions_data: List[Tuple[str, str]],
    tax_dict: Dict,
    loan_dict: Dict,
    pf_dict: Dict
) -> bool:
    """Validate all input data before processing."""
    try:
        required_employee_fields = [
            'employee_name', 'cnic', 'designation', 'department',
            'date_of_joining', 'employment_type', 'bank_name',
            'account_no', 'iban', 'employee', 'payable_to_employee'
        ]
        
        # Validate employee_data
        for field in required_employee_fields:
            if field not in employee_data:
                raise ValueError(f"Missing required field in employee_data: {field}")
        
        # Validate earnings_data and deductions_data structure
        if not isinstance(earnings_data, (list, tuple)):
            raise ValueError("earnings_data must be a list or tuple")
        if not isinstance(deductions_data, (list, tuple)):
            raise ValueError("deductions_data must be a list or tuple")
            
        # Validate data types in earnings and deductions
        for data in [earnings_data, deductions_data]:
            for item in data:
                if not isinstance(item, (list, tuple)) or len(item) != 2:
                    raise ValueError("Each item in earnings/deductions must be a tuple of (name, value)")
                
        # Validate tax_dict structure
        tax_fields = ['taxable_income', 'applicable_rate_for_exceeding_amount', 
                     'total_tax_chargeable', 'tax_already_paid', 'expected_monthly_deduction']
        validate_dictionary_structure(tax_dict, tax_fields, 'tax_dict')
        
        # Validate loan_dict structure if it contains non-zero amount
        if loan_dict.get('total_amount', 0) != 0:
            loan_fields = ['total_amount', 'paid_amount', 'remaining_amount',
                         'remaining_installments', 'loan_name']
            validate_dictionary_structure(loan_dict, loan_fields, 'loan_dict')
        
        # Validate pf_dict structure if it contains non-zero balance
        if pf_dict.get('current_balance', 0) != 0:
            validate_dictionary_structure(pf_dict, ['current_balance'], 'pf_dict')
        
        return True
    except Exception as e:
        logger.error(f"Validation error: {str(e)}")
        raise

def calculate_service_period(join_date: str) -> str:
    """Calculate service period in years, months, and days."""
    try:
        join_date = datetime.strptime(join_date, "%Y-%m-%d")
        today = datetime.now()
        difference = relativedelta(today, join_date)
        return f"{difference.years} Years, {difference.months} Months, {difference.days} Days"
    except Exception as e:
        logger.error(f"Error calculating service period: {str(e)}")
        return "Error calculating service period"

def create_payslip(
    employee_data: Dict,
    earnings_data: List[Tuple[str, str]],
    deductions_data: List[Tuple[str, str]],
    tax_dict: Dict,
    loan_dict: Dict,
    pf_dict: Dict,
    output_path: str
) -> str:
    """Create a payslip document with error handling and validation."""
    try:
        # Validate input data
        validate_input_data(
            employee_data,
            earnings_data,
            deductions_data,
            tax_dict,
            loan_dict,
            pf_dict
        )
        
        doc = Document()
        
        # Set margins
        for section in doc.sections:
            section.top_margin = section.bottom_margin = Inches(0.3)
            section.left_margin = section.right_margin = Inches(0.3)

        # Create header table
        header_table = doc.add_table(rows=1, cols=3)
        
        # Set column widths
        for i, width in enumerate([1.5, 3, 2]):
            header_table.columns[i].width = Inches(width)

        # Logo handling
        logo_cell = header_table.cell(0, 0)
        logo_paragraph = logo_cell.paragraphs[0]
        img_path = os.path.join('media', 'brb_logo', 'BRB-Logo.jpeg')
        
        try:
            logo_paragraph.add_run().add_picture(img_path, width=Inches(1.1))
        except Exception as e:
            logger.warning(f"Logo not found: {str(e)}")
            logo_paragraph.add_run("Logo not found")
        
        # Title and date
        center_cell = header_table.cell(0, 1)
        title_run = center_cell.paragraphs[0].add_run("BRB Group")
        title_run.bold = True
        title_run.font.size = Pt(16)
        center_cell.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        # Right cell formatting with proper date from employee data
        right_cell = header_table.cell(0, 2)
        right_para = right_cell.paragraphs[0]
        
        # Print date (current date)
        right_para.add_run(f"Print Date: {datetime.now().strftime('%B %d, %Y')}\n")
        
        # Pay statement text
        statement_run = right_para.add_run("PAY STATEMENT\n")
        statement_run.bold = True
        
        # Format the month from employee data (format: 'YYYY-MM' to 'Month YYYY')
        try:
            pay_month = datetime.strptime(employee_data['month'], '%Y-%m')
            formatted_month = pay_month.strftime('%B %Y')
            right_para.add_run(formatted_month)
        except (ValueError, KeyError) as e:
            logger.warning(f"Error formatting month: {str(e)}")
            right_para.add_run(employee_data.get('month', 'Month not specified'))
            
        right_para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        
        # Employee Information Table
        doc.add_paragraph()  # Spacing
        employee_table = doc.add_table(rows=8, cols=4)
        employee_table.style = 'Table Grid'
        
        # Fill employee information
        info_mapping = [
            ("Employee Name:", "employee_name", "Employee ID:", "employee"),
            ("Employment Type:", "employment_type", "CNIC:", "cnic"),
            ("Department:", "department", "Date of Joining:", "date_of_joining"),
            ("Bank:", "bank_name", "Bank Account No.:", "account_no"),
            ("", "", "Designation:", "designation"),
            ("", "", "Service Period:", "service_period"),
            ("", "", "IBAN:", "iban")
        ]
        
        for row_idx, (left_label, left_key, right_label, right_key) in enumerate(info_mapping):
            row = employee_table.rows[row_idx].cells
            row[0].text = left_label
            row[2].text = right_label
            
            if left_key:
                row[1].text = str(employee_data.get(left_key, "N/A"))
            if right_key == "service_period":
                row[3].text = calculate_service_period(employee_data["date_of_joining"])
            elif right_key:
                row[3].text = str(employee_data.get(right_key, "N/A"))

        # Add earnings and deductions tables
        doc.add_paragraph()
        earnings_table = doc.add_table(rows=1 + max(len(earnings_data), len(deductions_data)), cols=4)
        earnings_table.style = 'Table Grid'
        
        # Headers
        headers = earnings_table.rows[0].cells
        headers[0].text, headers[2].text = "Earnings", "Deductions"
        
        # Fill earnings and deductions
        for i, (name, value) in enumerate(earnings_data, 1):
            row = earnings_table.rows[i].cells
            row[0].text = str(name)
            row[1].text = str(value)
            
        for i, (name, value) in enumerate(deductions_data, 1):
            row = earnings_table.rows[i].cells
            row[2].text = str(name)
            row[3].text = str(value)

        # Add Tax Information Table if applicable
        if tax_dict and any(value != 0 for value in tax_dict.values()):
            doc.add_paragraph()
            tax_paragraph = doc.add_paragraph()
            tax_run = tax_paragraph.add_run("Income Tax [ if applicable]")
            tax_run.bold = True
            
            tax_table = doc.add_table(rows=2, cols=6)
            tax_table.style = 'Table Grid'
            
            # Tax headers
            tax_headers = [
                "Taxable Income", "Tax Slab", "Tax Credit | Adj.",
                "Tax Chargeable", "Tax Deducted", "Tax Payable"
            ]
            
            # Fill tax headers
            for idx, header in enumerate(tax_headers):
                cell = tax_table.rows[0].cells[idx]
                cell.text = header
                cell.paragraphs[0].runs[0].bold = True

            # Fill tax values
            tax_table.rows[1].cells[0].text = str(tax_dict["taxable_income"])
            tax_table.rows[1].cells[1].text = str(tax_dict["applicable_rate_for_exceeding_amount"])
            tax_table.rows[1].cells[2].text = str(tax_dict["tax_already_paid"])
            tax_table.rows[1].cells[3].text = str(tax_dict["total_tax_chargeable"])
            tax_table.rows[1].cells[4].text = str(tax_dict["tax_already_paid"])
            tax_table.rows[1].cells[5].text = str(tax_dict["expected_monthly_deduction"])

        # Add Provident Fund Table if applicable
        if pf_dict and pf_dict.get("current_balance", 0) != 0:
            doc.add_paragraph()
            pf_paragraph = doc.add_paragraph()
            pf_run = pf_paragraph.add_run("Provident Fund [ if applicable]")
            pf_run.bold = True
            
            pf_table = doc.add_table(rows=2, cols=5)
            pf_table.style = 'Table Grid'
            
            # PF headers
            pf_headers = [
                "Member Contribution", "Company Contribution",
                "Member Profit", "Company Profit", "Total Provident Fund"
            ]
            
            # Fill PF headers
            for idx, header in enumerate(pf_headers):
                cell = pf_table.rows[0].cells[idx]
                cell.text = header
                cell.paragraphs[0].runs[0].bold = True

            # Calculate and fill PF values
            current_balance = pf_dict["current_balance"]
            member_contribution = company_contribution = current_balance / 2
            
            pf_table.rows[1].cells[0].text = f"{member_contribution:,.2f}"
            pf_table.rows[1].cells[1].text = f"{company_contribution:,.2f}"
            pf_table.rows[1].cells[2].text = "0"
            pf_table.rows[1].cells[3].text = "0"
            pf_table.rows[1].cells[4].text = f"{current_balance:,.2f}"

        # Add Loan Details Table if applicable
        if loan_dict and loan_dict.get("total_amount", 0) != 0:
            doc.add_paragraph()
            loan_paragraph = doc.add_paragraph()
            loan_run = loan_paragraph.add_run("Loan Details [ if applicable]")
            loan_run.bold = True
            
            loan_table = doc.add_table(rows=2, cols=5)
            loan_table.style = 'Table Grid'
            
            # Loan headers
            loan_headers = [
                "Loan Name", "Opening", "Deduction",
                "Balance Amount", "Balance Installment"
            ]
            
            # Fill loan headers
            for idx, header in enumerate(loan_headers):
                cell = loan_table.rows[0].cells[idx]
                cell.text = header
                cell.paragraphs[0].runs[0].bold = True

            # Fill loan values
            loan_table.rows[1].cells[0].text = str(loan_dict["loan_name"])
            loan_table.rows[1].cells[1].text = f"{loan_dict['total_amount']:,.2f}"
            loan_table.rows[1].cells[2].text = f"{loan_dict['paid_amount']:,.2f}"
            loan_table.rows[1].cells[3].text = f"{loan_dict['remaining_amount']:,.2f}"
            loan_table.rows[1].cells[4].text = str(loan_dict["remaining_installments"])

        # Save document
        output_file = os.path.join(
            output_path,
            f"Pay_Slip_{employee_data['employee_name'].replace(' ', '_')}_{employee_data.get('month', 'current').replace(' ', '_')}.docx"
        )
        doc.save(output_file)
        logger.info(f"Payslip generated successfully: {output_file}")
        return output_file

    except Exception as e:
        error_msg = f"Error generating payslip: {str(e)}"
        logger.error(error_msg)
        raise Exception(error_msg)

# Usage example
if __name__ == "__main__":
    try:
        output_file = create_payslip(
            employee_data,
            earnings_data,
            deductions_data,
            tax_dict,
            loan_dict,
            pf_dict,
            "output_directory"
        )
        print(f"Payslip generated: {output_file}")
    except Exception as e:
        print(f"Failed to generate payslip: {str(e)}")

Error creating payslip: name 'Document' is not defined
